In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LinearRegression
import sys
sys.path.append("./models")
import importlib
import sparse_fwfm
importlib.reload(sparse_fwfm)
FwFM = sparse_fwfm.FwFM
TFwFM = sparse_fwfm.ThirdOrderFwFM

In [2]:
path = './data/'
df = pd.read_csv(path + "CrossDomain_2years.csv", index_col=0)
df.head()

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11,12,13,14,15,16,17,18,27,28,29,30,31,32,33,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,index,SEX,AGE,AREA,Salary,domein,job[0],job[1],job[2],job[3],...,importance[15],importance[16],importance[17],importance[18],Company,most_importance,question,sentence,factor,SCORE
0,0,男性,60,神奈川県,1000万円未満,1-1-18：医療保険,運送・輸送業,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,チューリッヒ生命,保障内容に対する保険料,Q2_1,申込み窓口の豊富さ（取扱い店舗、ネット申込み、保険営業員 等）,加入手続き,6
1,0,男性,60,神奈川県,1000万円未満,1-1-18：医療保険,運送・輸送業,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,チューリッヒ生命,保障内容に対する保険料,Q2_2,加入手続きの容易さ（記入事項の分かりやすさ、提出書類の内容、情報提供、等を含む）,加入手続き,6
2,0,男性,60,神奈川県,1000万円未満,1-1-18：医療保険,運送・輸送業,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,チューリッヒ生命,保障内容に対する保険料,Q2_3,加入手続き完了まで（契約完了まで）のスピード,加入手続き,6
3,0,男性,60,神奈川県,1000万円未満,1-1-18：医療保険,運送・輸送業,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,チューリッヒ生命,保障内容に対する保険料,Q2_4,商品内容の分かりやすさ,商品内容,6
4,0,男性,60,神奈川県,1000万円未満,1-1-18：医療保険,運送・輸送業,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,チューリッヒ生命,保障内容に対する保険料,Q2_5,商品内容の充実度（特約等を含む）,商品内容,6


In [3]:
df_count = df.groupby(["domein", "Company"])["index"].count().reset_index().rename(columns = {"index":"count"})
df = pd.merge(df, df_count, on=["domein", "Company"])
df = df[df["count"] > 100]#.drop("count",axis=1)

In [4]:
df_X = df[["SEX", "AGE", "Salary", "domein", "job[0]", "job[1]", "importance[0]", "importance[1]", "Company", "question", "sentence", "factor"]].fillna("Nan").copy()
df_y = df["SCORE"].copy()

# 2nd Order

In [17]:
df_test_eval = pd.DataFrame({"Company":[],"factor":[],"domein":[],"fwfm_score":[], "linear_score":[], "SCORE":[]})
group_kfold = GroupKFold(n_splits=3)
userid = df["index"].values
for train_index, test_index in group_kfold.split(df_X, df_y, userid):
    df_X_train, df_y_train = df_X.iloc[train_index], df_y.iloc[train_index]
    df_X_test, df_y_test = df_X.iloc[test_index], df_y.iloc[test_index]
    
    #sample_weight = np.clip(1 / df["count"].iloc[train_index].values, 0, 10) + 1e-3
    #sample_weight = (sample_weight - sample_weight.min()) / (sample_weight.max() - sample_weight.min())
    
    mean = df_y_train.mean()
    std = df_y_train.std()
    df_y_train_std = (df_y_train - mean) / std
    fwfm = FwFM(
        df_X_train, df_y_train_std, 
        #sample_weight = sample_weight, 
        n_epoch=5, dim=16, n_batch=1024,
        optimizer = "Adam",
        lam_r = 1e-3,
    )
    y_hat = fwfm(df_X_test) * std + mean
    fwfm_score = mean_squared_error(df_y_test.values, y_hat) ** 0.5

    reg = LinearRegression()
    reg.fit(fwfm.X, df_y_train.values)
    X_test = fwfm.convert_sparse(df_X_test)
    y_hat_linear = reg.predict(X_test)
    linear_score = mean_squared_error(df_y_test.values, y_hat_linear) ** 0.5

    df_X_test_eval = df_X_test[["Company","factor","domein"]].copy()
    df_X_test_eval["fwfm_score"] = y_hat
    df_X_test_eval["linear_score"] = y_hat_linear
    df_X_test_eval["SCORE"] = df_y_test

    df_test_eval = pd.concat([df_test_eval, df_X_test_eval])

    print(f"fwfm = {fwfm_score}, linear = {linear_score}") 
    break 

100%|██████████| 84/84 [00:00<00:00, 102.70it/s]


fwfm = 1.7642699693756483, linear = 1.7629018910462757


In [16]:
df_eval_gby = df_test_eval.groupby(["Company","factor","domein"])[["SCORE","fwfm_score", "linear_score"]].mean().reset_index()
df_eval_gby["fwfm_squared_error"] = (df_eval_gby["fwfm_score"] - df_eval_gby["SCORE"]) ** 2
df_eval_gby["linear_squared_error"] = (df_eval_gby["linear_score"] - df_eval_gby["SCORE"]) ** 2
df_eval_gby["fwfm_root_mean_squared_error"] = df_eval_gby["fwfm_squared_error"] ** 0.5
df_eval_gby["linear_root_mean_squared_error"] = df_eval_gby["linear_squared_error"] ** 0.5

df_eval = df_eval_gby.groupby("domein").mean(["fwfm_squared_error", "linear_squared_error"]).reset_index()[["domein", "fwfm_root_mean_squared_error", "linear_root_mean_squared_error"]]
display(df_eval)
df_eval.mean(axis=0)

,domein,fwfm_root_mean_squared_error,linear_root_mean_squared_error
0,1-1-18：医療保険,0.386670,0.348105
1,1-1-19：医療保険,0.359107,0.366350
2,1-2-17：海外旅行保険,0.373706,0.372760
3,1-2-18：海外旅行保険,0.444648,0.386199
4,1-4-18：がん保険,0.432457,0.451401
5,1-4-19：がん保険,0.782821,0.794671
6,1-7-18：生命保険,0.340872,0.342405
7,1-7-19：生命保険,0.350139,0.355950
8,1-9-17：ペット保険,0.480824,0.558250
9,1-9-18：ペット保険,0.537070,0.521644


fwfm_root_mean_squared_error      0.448831
linear_root_mean_squared_error    0.449773
dtype: float64

# 3rd Order 

In [5]:
df_test_eval = pd.DataFrame({"Company":[],"factor":[],"domein":[],"fwfm_score":[], "linear_score":[], "SCORE":[]})
group_kfold = GroupKFold(n_splits=3)
userid = df["index"].values
for train_index, test_index in group_kfold.split(df_X, df_y, userid):
    df_X_train, df_y_train = df_X.iloc[train_index], df_y.iloc[train_index]
    df_X_test, df_y_test = df_X.iloc[test_index], df_y.iloc[test_index]
    
    #sample_weight = np.clip(1 / df["count"].iloc[train_index].values, 0, 10) + 1e-3
    #sample_weight = (sample_weight - sample_weight.min()) / (sample_weight.max() - sample_weight.min())
    
    mean = df_y_train.mean()
    std = df_y_train.std()
    df_y_train_std = (df_y_train - mean) / std
    fwfm = TFwFM(
        df_X_train, df_y_train_std, 
        #sample_weight = sample_weight, 
        n_epoch=5, dim=16, n_batch=256
    )
    y_hat = fwfm(df_X_test) * std + mean
    fwfm_score = mean_squared_error(df_y_test.values, y_hat) ** 0.5

    reg = LinearRegression()
    reg.fit(fwfm.X, df_y_train.values)
    X_test = fwfm.convert_sparse(df_X_test)
    y_hat_linear = reg.predict(X_test)
    linear_score = mean_squared_error(df_y_test.values, y_hat_linear) ** 0.5

    df_X_test_eval = df_X_test[["Company","factor","domein"]].copy()
    df_X_test_eval["fwfm_score"] = y_hat
    df_X_test_eval["linear_score"] = y_hat_linear
    df_X_test_eval["SCORE"] = df_y_test

    df_test_eval = pd.concat([df_test_eval, df_X_test_eval])

    print(f"fwfm = {fwfm_score}, linear = {linear_score}") 

  2%|▏         | 15/679 [00:57<42:29,  3.84s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-f58940cff8f7>", line 10, in <module>
    fwfm = TFwFM(
  File "./models/sparse_fwfm.py", line 336, in __init__
    self.train()
  File "./models/sparse_fwfm.py", line 455, in train
    dL_dv = (a * self.der_v(X_batch, reduction=None)).mean(axis=2)
  File "./models/sparse_fwfm.py", line 490, in der_v
    dv_3rd = self.contract_der_v_3rd(
  File "/opt/conda/lib/python3.8/site-packages/opt_einsum/contract.py", line 763, in __call__
    return self._contract(ops, out, backend, evaluate_constants=evaluate_constants)
  File "/opt/conda/lib/python3.8/site-packages/opt_einsum/contract.py", line 693, in _contract
    return _core_contract(list(arrays),
  File "/opt/conda/lib/python3.8/site-packages/opt_einsum/contract.py", line 591, in _core_contract
    new_view = _einsum(einsum

TypeError: object of type 'NoneType' has no len()

In [ ]:
df_eval_gby = df_test_eval.groupby(["Company","factor","domein"])[["SCORE","fwfm_score", "linear_score"]].mean().reset_index()
df_eval_gby["fwfm_squared_error"] = (df_eval_gby["fwfm_score"] - df_eval_gby["SCORE"]) ** 2
df_eval_gby["linear_squared_error"] = (df_eval_gby["linear_score"] - df_eval_gby["SCORE"]) ** 2
df_eval_gby["fwfm_root_mean_squared_error"] = df_eval_gby["fwfm_squared_error"] ** 0.5
df_eval_gby["linear_root_mean_squared_error"] = df_eval_gby["linear_squared_error"] ** 0.5

df_eval = df_eval_gby.groupby("domein").mean(["fwfm_squared_error", "linear_squared_error"]).reset_index()[["domein", "fwfm_root_mean_squared_error", "linear_root_mean_squared_error"]]
display(df_eval)
df_eval.mean(axis=0)